In [1]:
import pandas as pd
from datasets import load_dataset
from google.cloud import translate
from tqdm import tqdm

In [ ]:
def translate_texts(texts, project_id="id-csqa", src_lang="en", tgt_lang="id"):

    client = translate.TranslationServiceClient()
    location = "global"
    parent = f"projects/{project_id}/locations/{location}"

    response = client.translate_text(
        request={
            "parent": parent,
            "contents": texts,
            "mime_type": "text/plain",
            "source_language_code": src_lang,
            "target_language_code": tgt_lang,
        }
    )

    return [t.translated_text for t in response.translations]

In [ ]:
comqa_train = load_dataset('commonsense_qa', cache_dir='/mnt/nas2/kikiputri/cache', split='train')
comqa_valid = load_dataset('commonsense_qa', cache_dir='/mnt/nas2/kikiputri/cache', split='validation')

In [ ]:
comqa_valid[0]

In [ ]:
len(comqa_valid)

In [ ]:
comqa_trans = []
tgt_lang = 'su'
# i = 0
for item in tqdm(comqa_valid):
    # i += 1
    # if i <= 2558:
    #     continue
    trans_items = [item['question'], item['question_concept']] + item['choices']['text']
    trans_texts = translate_texts(trans_items, tgt_lang=tgt_lang)
    comqa_trans.append({
        'id': item['id'],
        'question': trans_texts[0],
        'question_concept': trans_texts[1],
        'choices': {
            'label': item['choices']['label'],
            'text': trans_texts[2:]
        },
        'answerKey': item['answerKey']
    })

In [ ]:
len(comqa_trans)

In [ ]:
comqa_trans_df = pd.DataFrame(comqa_trans)

In [ ]:
comqa_trans_df

In [ ]:
# WARNING!!!! make sure the data type is correct
filename = 'valid_gtrans_su'

In [ ]:
comqa_trans_df.to_csv('/mnt/nas2/kikiputri/indo-local-transfer/datasets/csqa/' + filename  + '.csv', index=False)